In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import glob
import sys

sys.path.insert(0,'../../..')
from sed_endtoend.pcen.model import SMel_PCEN, concatenate_stft
from sed_endtoend.cnn.model import build_custom_cnn

from params import *

os.environ["CUDA_VISIBLE_DEVICES"]="1"

# files parameters
Nfiles = None
resume = False
load_subset = Nfiles

Using TensorFlow backend.


In [13]:
sequence_frames = 44

model_mel = SMel_PCEN(mel_bands,sequence_frames,audio_win,audio_hop)    
model_cnn = build_custom_cnn(n_freq_cnn=mel_bands, n_frames_cnn=sequence_frames,large_cnn=large_cnn)

model = concatenate_stft(sequence_frames,513,model_cnn,model_mel)

model.load_weights("../03_train_SMel_CNN/weights_best.hdf5")

model_mel.summary()

filters = model_mel.layers[1].get_weights()[0]
print(filters.shape)

b = model_mel.layers[2].get_weights()[0]
print(b.shape)

[alpha, delta, r] = model_mel.layers[3].get_weights()
print(alpha.shape)
print(delta.shape)
print(r.shape)

np.save('filters.npy',filters)
np.save('b.npy',b)
np.save('alpha.npy',alpha)
np.save('delta.npy', delta)
np.save('r.npy',r)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           (None, 44, 513)      0                                            
__________________________________________________________________________________________________
dot_12 (DOT)                    (None, 44, 128)      65664       input_34[0][0]                   
__________________________________________________________________________________________________
rnn_12 (RNN)                    (None, 44, 128)      128         dot_12[0][0]                     
__________________________________________________________________________________________________
pcen_12 (PCEN)                  (None, 44, 128)      384         dot_12[0][0]                     
                                                                 rnn_12[0][0]                     
Total para